In [23]:
!pip install pandas pandas-ta numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt requests requests_cache requests_ratelimiter pyrate_limiter


In [27]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

warnings.filterwarnings('ignore')

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
session.headers['User-agent'] = 'my-program/1.0'
session

<CachedSession(cache=<SQLiteCache(name=http_cache)>, settings=CacheSettings(expire_after=-1))>

In [34]:
ticker = yf.Ticker('spy', session=session)
# The scraped response will be stored in the cache
# ticker.actions

df = yf.download("SPY LLY", period="5y")

[*********************100%%**********************]  2 of 2 completed


In [30]:
hist = ticker.history(period="10y")

hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2014-05-27 00:00:00-04:00,159.670698,160.105271,159.578769,160.055130,72010000,0.0,0.0,0.0
2014-05-28 00:00:00-04:00,160.055048,160.305763,159.670616,159.938049,66723000,0.0,0.0,0.0
2014-05-29 00:00:00-04:00,160.305812,160.790512,159.896310,160.765442,64377000,0.0,0.0,0.0
2014-05-30 00:00:00-04:00,160.615047,161.124830,160.481330,161.024536,76316000,0.0,0.0,0.0
2014-06-02 00:00:00-04:00,161.250162,161.283598,160.431170,161.208374,64656000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-05-20 00:00:00-04:00,529.570007,531.559998,529.169983,530.059998,37764200,0.0,0.0,0.0
2024-05-21 00:00:00-04:00,529.280029,531.520020,529.070007,531.359985,33437000,0.0,0.0,0.0
2024-05-22 00:00:00-04:00,530.650024,531.380005,527.599976,529.830017,48390000,0.0,0.0,0.0


In [33]:
hist['3-day Change'] = hist['Close'].pct_change(periods=3) * 100

drop_days = hist[hist['3-day Change'] < -1]

drop_days

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,3-day Change
Date,,,,,,,,,
2014-06-12 00:00:00-04:00,162.704237,162.796166,161.383815,161.743164,106350000,0.0,0.0,0.0,-1.043081
2014-07-31 00:00:00-04:00,164.256433,164.399183,162.039589,162.140350,183479000,0.0,0.0,0.0,-2.381208
2014-08-01 00:00:00-04:00,161.695324,162.702978,160.864015,161.644943,189261000,0.0,0.0,0.0,-2.259455
2014-08-04 00:00:00-04:00,161.955592,163.156389,161.267033,162.812103,91340000,0.0,0.0,0.0,-1.568721
2014-08-07 00:00:00-04:00,162.014425,162.173973,160.007509,160.410568,135733000,0.0,0.0,0.0,-1.475035
...,...,...,...,...,...,...,...,...,...
2024-04-15 00:00:00-04:00,515.130005,515.299988,503.579987,504.450012,92101400,0.0,0.0,0.0,-1.880881
2024-04-16 00:00:00-04:00,504.940002,506.500000,502.209991,503.529999,73484000,0.0,0.0,0.0,-2.793437
2024-04-17 00:00:00-04:00,506.049988,506.220001,499.119995,500.549988,75910300,0.0,0.0,0.0,-2.016251


In [17]:
# end_date = '2024-05-23'
# # print(symbols_list)
# symbols_list = ['SPY', 'QQQ']

# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list,
#                  start=start_date,
#                  end=end_date).stack()


# df.index.names = ['date', 'ticker']

# df.columns = df.columns.str.lower()

# df.set_index('Date', inplace=True)

# df.index.names = ['date', 'ticker']

# df['3daychange'] = df['Close'].pct_change(periods=3) * 100

# # drop_days = df[df['3daychange'] < -2]

# df

[*********************100%%**********************]  2 of 2 completed


Price               Adj Close       Close        High         Low        Open  \
date       ticker                                                               
2016-05-25 QQQ     102.894539  109.239998  109.570000  108.680000  108.870003   
           SPY     182.129211  209.279999  209.770004  207.869995  208.669998   
2016-05-26 QQQ     103.195969  109.559998  109.699997  109.089996  109.349998   
           SPY     182.181473  209.339996  209.710007  208.970001  209.440002   
2016-05-27 QQQ     103.732841  110.129997  110.139999  109.610001  109.629997   
...                       ...         ...         ...         ...         ...   
2024-05-20 SPY     530.059998  530.059998  531.559998  529.169983  529.570007   
2024-05-21 QQQ     455.799988  455.799988  455.989990  452.839996  453.040009   
           SPY     531.359985  531.359985  531.520020  529.070007  529.280029   
2024-05-22 QQQ     455.709991  455.709991  456.820007  453.079987  456.059998   
           SPY     529.830017  529.830017  531.380005  527.599976  530.650024   

Price                Volume  3daychange  
date       ticker                        
2016-05-25 QQQ     37951700         NaN  
           SPY     76621400         NaN  
2016-05-26 QQQ     21885900         NaN  
           SPY     55280700   91.633102  
2016-05-27 QQQ     19870000  -47.376721  
...                     ...         ...  
2024-05-20 SPY     37764200   17.332209  
2024-05-21 QQQ     23167700  -13.910666  
           SPY     33437000   16.805518  
2024-05-22 QQQ     25120000  -14.026715  
           SPY     48390000   16.241780  

[4024 rows x 7 columns]

In [38]:
symbols_list = ['SPY', 'LLY', 'AMZN']

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date, session=session).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  3 of 3 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-25 AMZN     35.417500   35.417500   35.542999   35.276001   35.400002   
           LLY      64.313866   74.540001   75.500000   74.449997   75.489998   
           SPY     182.129211  209.279999  209.770004  207.869995  208.669998   
2016-05-26 AMZN     35.745499   35.745499   35.750000   35.364498   35.416500   
           LLY      65.159447   75.519997   75.599998   74.269997   74.599998   
...                       ...         ...         ...         ...         ...   
2024-05-21 LLY     803.169983  803.169983  816.609985  791.390015  792.460022   
           SPY     531.359985  531.359985  531.520020  529.070007  529.280029   
2024-05-22 AMZN    183.130005  183.130005  185.220001  181.970001  183.880005   
           LLY     802.909973  802.909973  804.880005  795.250000  801.000000   
           SPY     529.830017  529.830017  531.380005  527.599976  530.650024   

Price                volume  
date       ticker            
2016-05-25 AMZN    65354000  
           LLY      4621700  
           SPY     76621400  
2016-05-26 AMZN    48934000  
           LLY      2654600  
...                     ...  
2024-05-21 LLY      4334500  
           SPY     33437000  
2024-05-22 AMZN    28148800  
           LLY      1767500  
           SPY     48390000  

[6036 rows x 6 columns]

In [54]:
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-25 AMZN     35.417500   35.417500   35.542999   35.276001   35.400002   
           LLY      64.313866   74.540001   75.500000   74.449997   75.489998   
           SPY     182.129211  209.279999  209.770004  207.869995  208.669998   
2016-05-26 AMZN     35.745499   35.745499   35.750000   35.364498   35.416500   
           LLY      65.159447   75.519997   75.599998   74.269997   74.599998   
...                       ...         ...         ...         ...         ...   
2024-05-21 LLY     803.169983  803.169983  816.609985  791.390015  792.460022   
           SPY     531.359985  531.359985  531.520020  529.070007  529.280029   
2024-05-22 AMZN    183.130005  183.130005  185.220001  181.970001  183.880005   
           LLY     802.909973  802.909973  804.880005  795.250000  801.000000   
           SPY     529.830017  529.830017  531.380005  527.599976  530.650024   

Price                volume        rsi      macd  dollar_volume  
date       ticker                                                
2016-05-25 AMZN    65354000        NaN       NaN    2314.675265  
           LLY      4621700        NaN       NaN     297.239393  
           SPY     76621400        NaN       NaN   13954.995160  
2016-05-26 AMZN    48934000        NaN       NaN    1749.170231  
           LLY      2654600        NaN       NaN     172.972267  
...                     ...        ...       ...            ...  
2024-05-21 LLY      4334500  60.015516  1.048868    3481.340291  
           SPY     33437000  65.885280  1.209717   17767.083830  
2024-05-22 AMZN    28148800  51.438055  0.161655    5154.889881  
           LLY      1767500  59.943123  1.341245    1419.143378  
           SPY     48390000  63.954088  1.225348   25638.474527  

[6036 rows x 9 columns]

In [74]:
spy = df.xs('SPY', level=1, drop_level=False)


spy['3daychange'] = spy['close'].pct_change(periods=3) * 100

spy[spy['3daychange'] < -1]

,Price,adj close,close,high,low,open,volume,rsi,macd,dollar_volume,3daychange
date,ticker,,,,,,,,,,
2016-06-13,SPY,181.406906,208.449997,210.369995,208.350006,209.360001,117751200,NaN,NaN,21360.880885,-1.845834
2016-06-14,SPY,181.050140,208.039993,208.740005,206.919998,208.000000,125059300,NaN,NaN,22642.003821,-1.904946
2016-06-15,SPY,180.797775,207.750000,209.360001,207.529999,208.039993,109124500,NaN,NaN,19729.466827,-1.104397
2016-06-24,SPY,177.792618,203.240005,210.850006,202.720001,203.630005,333444400,35.552493,NaN,59283.952766,-2.494721
2016-06-27,SPY,174.608429,199.600006,201.600006,198.649994,201.589996,230775800,29.387883,NaN,40295.399879,-4.084575
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-15,SPY,504.450012,504.450012,515.299988,503.579987,515.130005,92101400,44.820312,-0.091312,46460.552354,-1.880881
2024-04-16,SPY,503.529999,503.529999,506.500000,502.209991,504.940002,73484000,44.131039,-0.350318,37001.398430,-2.793437
2024-04-17,SPY,500.549988,500.549988,506.220001,499.119995,506.049988,75910300,41.932341,-0.616858,37996.899738,-2.016251
